<a href="https://colab.research.google.com/github/kinking2008/Workspace/blob/master/%E5%B0%8F%E8%AF%B4%E4%B8%8B%E8%BD%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir nn
%cd /content/drive/My Drive/nn/


In [ ]:
#@title 开始下载
# coding: utf-8
import requests
import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup
import time

#大写转小写
CN_NUM = {
    '〇': 0, '一': 1, '二': 2, '三': 3, '四': 4, '五': 5, '六': 6, '七': 7, '八': 8, '九': 9, '零': 0,
    '壹': 1, '贰': 2, '叁': 3, '肆': 4, '伍': 5, '陆': 6, '柒': 7, '捌': 8, '玖': 9, '貮': 2, '两': 2,
}

CN_UNIT = {
    '十': 10,
    '拾': 10,
    '百': 100,
    '佰': 100,
    '千': 1000,
    '仟': 1000,
    '万': 10000,
    '萬': 10000,
    '亿': 100000000,
    '億': 100000000,
    '兆': 1000000000000,
}

# 调用大写转换
def chinese_to_arabic(cn:str) -> int:
    if str.isdigit(cn):
        val=int(cn)
    else:
        unit = 0   # current
        ldig = []  # digest
        for cndig in reversed(cn):
            if cndig in CN_UNIT:
                unit = CN_UNIT.get(cndig)
                if unit == 10000 or unit == 100000000:
                    ldig.append(unit)
                    unit = 1
            else:
                dig = CN_NUM.get(cndig)
                if unit:
                    dig *= unit
                    unit = 0
                ldig.append(dig)
        if unit == 10:
            ldig.append(10)
        val, tmp = 0, 0
        for x in reversed(ldig):
            if x == 10000 or x == 100000000:
                val += tmp * x
                tmp = 0
            else:
                tmp += x
        val += tmp
    return val
#/大写转小写


headers = {
    # 'referer': 'https://www.63xs.com/',
    # 'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
}

#把每章节的内容添加到目录的当前的小说文本中
def down(url,name,bookname,path):
    html_2 = BeautifulSoup(requests.get(url,verify=False).content,"lxml")
    book=html_2.find_all(id = "cContent")[0].get_text()
    book = book.replace('　　', '\n')
    book = book.replace('\xa0\xa0\xa0\xa0', '      ')
    book = book.replace('\n\r\n', '\n')
    book = name + '\n\n' + book + '\n\n\n\n'
    with open(path + "/" + bookname + ".txt", 'a', encoding='utf-8') as f:
        f.write(book)

#判断临时页存文件在不在，不存在就新建一个，读取报错就返回mdr1=0
def except_strip(m2):
    try:
        f3= open(path + "/" + m2+".txt", 'r', encoding='utf-8')
        mdr1 = f3.readline()
        f3.close()
        return mdr1
    except:

        mdr1 = 0
        return mdr1


#url = input ('请输入书籍链接')
#path = input ('请输入要保存的路径')
if __name__ == '__main__':
    url = 'http://ewenxue.org/xs/196765/mulu.htm'
    path='/content/drive/My Drive/nn'
    html = BeautifulSoup(requests.get(url,headers = headers,verify=False).content,"lxml")
    boname = html.find_all('meta')[0]['content']
    bookname=boname[:boname.index("，")]
    print ('开始下载' + bookname)
    html_1 = html.find_all('ul')[2]
    html_6=html_1.find_all('li')
    m2="m1"
    #调用excpt_strip取得上次下完的页码
    mdr1=except_strip(m2)

    html_7 = html_6[int(mdr1):]
    for i in html_7:
        c = 'http://ewenxue.org/xs/196765/' + i.a['href']
        #格式化章节名  搜，号替换成"章"字
        name = "第"+i.get_text().replace("，", "章", 1);
        down(c,name,bookname,path)
        print ('已下载' + name)

        #把临时页存入目录下的M1文件中
        x=i.get_text().replace("，", "章", 1)[0:4]
        with open(path + "/" + m2+".txt", 'w', encoding='utf-8') as f4:
            f4.write(x)
            f4.close()
    print( x + "章，已下载完成")

